# Importing Dependencies

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import os
import string
from collections import Counter
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Creating the custom STOPWORD dictionary

In [ ]:
# initialise a string to save the txt
stop_string = ''

dir = '/content/drive/MyDrive/Internship/StopWords'

dir_cnt = 0    

# preprocessing and saving the text in the directories in a list named stop_words
for filename in os.listdir(dir) :

    f = os.path.join(dir, filename)
    with open(f, 'r', encoding = 'latin') as text_file :
      dir_cnt += 1

      content = text_file.read()
      content = content.replace('\n', ' ')
      content = content.replace('|', ' ')

      stop_string += content
      stop_words = stop_string.split(' ')

print(stop_words)
print(f'The total stop words are {len(stop_words)} in total of {dir_cnt} ".txt" files.')

In [ ]:
print(f"Number of null Strings are {stop_words.count('')}.")

# removing any null values
stop_words = [item for item in stop_words if item != '']

print(f'The total Stop Words now is {len(stop_words)} after removing the null strings.')

Number of null Strings are 311.
The total Stop Words now is 14033 after removing the null strings.


In [ ]:
stop_words = sorted( stop_words, key = len, reverse = True )

# removing the links
stop_words = [item for item in stop_words if 'http' not in item]
stop_words = [item for item in stop_words if 'gov' not in item]

stop_words = sorted( stop_words, key = len, reverse = True )
print(stop_words)

In [ ]:
print(f'Number of stop words are {len(stop_words)}.\n', stop_words)

# Positive and Negative Dictionary

In [ ]:
# Making a positive dict
positive_file = open('/content/drive/MyDrive/Internship/MasterDictionary/positive-words.txt', 'r', encoding = 'latin')
positive_string = positive_file.read()
positive_string = positive_string.replace('\n', ' ')
positive_string = positive_string.replace('|', ' ')

positive_dict = positive_string.split(' ')
positive_dict.remove('')

# Making a negative dict
negative_file = open('/content/drive/MyDrive/Internship/MasterDictionary/negative-words.txt', 'r', encoding = 'latin')
negative_string = negative_file.read()
negative_string = negative_string.replace('\n', ' ')
negative_string = negative_string.replace('|', ' ')

negative_dict = negative_string.split(' ')
negative_dict.remove('')

print(positive_dict.count(''), positive_dict)
print(negative_dict.count(''), negative_dict)

In [ ]:
output = pd.read_excel('/content/drive/MyDrive/Internship/Output Data Structure.xlsx')
output.head()

# Web-Scrapping and the Main Script

In [ ]:
# Function to Scrap the tag text from soup
def get_data(soup):
    try:
      title = soup.find('h1').text
      
      try :
        div_element = soup.find_all('div', {'class' : 'tdb-block-inner td-fix-index'})
        p_tags = div_element[14].find_all('p')

      except Exception as e : # Inconsistent webpage formatting, there are two types of format in the given links
        div_element = soup.find_all('div', {'class' : 'td-post-content tagdiv-type'})
        p_tags = div_element[0].find_all('p') 

      content = ''
      for p in p_tags:
        content += p.text

    except Exception as  e:
      return e

    return title, content

# function to count the complex word in a string of text
def get_complex(content):
  cnt = 0
  for wrd in content :
    if len(wrd) > 2 : 
      cnt += 1
  return cnt

# function to calculate syllabal count handling "US"
def get_syb(word_list):
  vowels = 'aeiou'
  cnt = 0
  prev_char = None
  
  for word in word_list:
    for char in word:
      if char in vowels and (prev_char is None or prev_char not in vowels):
        cnt += 1
      if word[-2:] == 'es' or word[-2:] == 'ed':
        cnt -= 1
  return cnt
    
if __name__ == '__main__':

    HEADERS = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36', 'Acccept-Language': 'en-US, en; q = 0.5'})

    output = pd.read_excel('/content/drive/MyDrive/Internship/Output Data Structure.xlsx')

    # Looping over all the URL
    for url in output['URL']:
        
        # hitting the URLs with requests
        webpage = requests.get(url, headers = HEADERS)
        # parsing the response
        soup = BeautifulSoup(webpage.content, 'html.parser')

        try:
            # get the title and content
            title, content = get_data(soup)
            
            # combine them
            content = title + ' ' + content
            
            # save the original text, will be needed down the line
            txt = content
            
            # save the number of fullstops in the original txt
            fullstop_cnt = content.count('.')
            
            # split the content by space as a delimitor
            content = content.split(' ')
            
            # lowercase cast all the words
            content = [word.lower() for word in content]

            # cleaning the content with RESPECT TO THE CUSTOM STOPWORDS (as requested) 
            cleaned_content = list(set(content) - set(stop_words))

            # calculating positive score
            positive_scr = len(set(content).intersection(positive_dict))
            output.loc[output['URL'] == url, 'POSITIVE SCORE'] = positive_scr
            
            # calculating negative score
            negative_scr = len(set(content).intersection(negative_dict))
            output.loc[output['URL'] == url, 'NEGATIVE SCORE'] = negative_scr

            # calculating polarity score
            Polarity_Score = (positive_scr - negative_scr) / ((positive_scr + negative_scr) + 0.000001)
            output.loc[output['URL'] == url, 'POLARITY SCORE'] = Polarity_Score

            # calculating subjectivity score
            subjectivity_score = (positive_scr + negative_scr )/ ( len(cleaned_content) + 0.000001)
            output.loc[output['URL'] == url, 'SUBJECTIVITY SCORE'] = subjectivity_score

            # calculating average sentence length 
            avg_sentence_len = len(cleaned_content) / fullstop_cnt
            output.loc[output['URL'] == url, 'AVG SENTENCE LENGTH'] = avg_sentence_len

            # calculating complex word count
            complex_word_cnt = get_complex(cleaned_content)
            output.loc[output['URL'] == url, 'COMPLEX WORD COUNT'] = complex_word_cnt

            # calculating complex word percentage
            complex_word_per = (complex_word_cnt / len(cleaned_content)) * 100
            output.loc[output['URL'] == url, 'PERCENTAGE OF COMPLEX WORDS'] = complex_word_per
            
            # calculating fog index
            fog_idx = 0.4 * ( avg_sentence_len + complex_word_per )
            output.loc[output['URL'] == url, 'FOG INDEX'] = fog_idx

            # calculating the words per sentence
            words_per_sent = len(cleaned_content) / fullstop_cnt
            output.loc[output['URL'] == url, 'AVG NUMBER OF WORDS PER SENTENCE'] = words_per_sent
            
            
            ## processing the stop words from nltk
            nltk_stp = set(stopwords.words('english'))
            nltk_stp |= set(string.punctuation)
            nltk_stp = [word.lower() for word in nltk_stp]
            filtered_list = [word for word in cleaned_content if word not in nltk_stp]
            word_count = len(filtered_list)
            output.loc[output['URL'] == url, 'WORD COUNT'] = word_count

            ## syllable count per word
            syb_cnt = get_syb(filtered_list) / len(filtered_list)
            output.loc[output['URL'] == url, 'SYLLABLE PER WORD'] = syb_cnt

            ## Personal Pronouns
            count_US = txt.count('US')
            txt = txt.lower()
            pronoun_cnt = txt.count('I') + txt.count('we') + txt.count('ours') + txt.count('my') + txt.count('us') - 2*count_US
            output.loc[output['URL'] == url, 'PERSONAL PRONOUNS'] = pronoun_cnt

            ## Average Word Length
            avg_word_length = len(txt) / len(cleaned_content)
            output.loc[output['URL'] == url, 'AVG WORD LENGTH'] = avg_word_length

        except Exception as e:
          print(f' The following webpage does not exist for {url}') # 3 instances of webpage not founds or error 404 
 

# Final App.py Script

In [ ]:
# Import the dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import os
import string
from collections import Counter
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# function for generating all the dicts
def get_stop_words(dir):
  # initialise a string to save the txt
    stop_string = ''

    dir_cnt = 0    

    # preprocessing and saving the text in the directories in a list named stop_words
    for filename in os.listdir(dir) :

        f = os.path.join(dir, filename)
        with open(f, 'r', encoding = 'latin') as text_file :
          dir_cnt += 1

          content = text_file.read()
          content = content.replace('\n', ' ')
          content = content.replace('|', ' ')

          stop_string += content
          stop_words = stop_string.split(' ')

    # removing any null values
    stop_words = [item for item in stop_words if item != '']
    stop_words = sorted( stop_words, key = len, reverse = True )

    # removing the links
    stop_words = [item for item in stop_words if 'http' not in item]
    stop_words = [item for item in stop_words if 'gov' not in item]

    return stop_words

# function for generating positive and negative dicts
def get_dicts(pos_path, neg_path):
      # Making a positive dict
      positive_file = open(pos_path, 'r', encoding = 'latin')
      positive_string = positive_file.read()
      positive_string = positive_string.replace('\n', ' ')
      positive_string = positive_string.replace('|', ' ')

      positive_dict = positive_string.split(' ')
      positive_dict.remove('')

      # Making a negative dict
      negative_file = open(neg_path, 'r', encoding = 'latin')
      negative_string = negative_file.read()
      negative_string = negative_string.replace('\n', ' ')
      negative_string = negative_string.replace('|', ' ')

      negative_dict = negative_string.split(' ')
      negative_dict.remove('')

      return positive_dict, negative_dict

# Function to Scrap the tag text from soup
def get_data(soup):
    try:
      title = soup.find('h1').text
      
      try :
        div_element = soup.find_all('div', {'class' : 'tdb-block-inner td-fix-index'})
        p_tags = div_element[14].find_all('p')

      except Exception as e : # Inconsistent webpage formatting, there are two types of format in the given links
        div_element = soup.find_all('div', {'class' : 'td-post-content tagdiv-type'})
        p_tags = div_element[0].find_all('p') 

      content = ''
      for p in p_tags:
        content += p.text

    except Exception as  e:
      print(e)

    return title, content

# function to count the complex word in a string of text
def get_complex(content):
  cnt = 0
  for wrd in content :
    if len(wrd) > 2 : 
      cnt += 1
  return cnt

# function to calculate syllabal count, handling "US"
def get_syb(word_list):
  vowels = 'aeiou'
  cnt = 0
  prev_char = None
  
  for word in word_list:
    for char in word:
      if char in vowels and (prev_char is None or prev_char not in vowels):
        cnt += 1
      if word[-2:] == 'es' or word[-2:] == 'ed':
        cnt -= 1
  return cnt
    
if __name__ == '__main__':

    # change your directories here
    stop_dir = '/content/drive/MyDrive/Internship/StopWords'
    positive_dir =  '/content/drive/MyDrive/Internship/MasterDictionary/positive-words.txt'
    negative_dir = '/content/drive/MyDrive/Internship/MasterDictionary/negative-words.txt'

    stop_words = get_stop_words(stop_dir)
    positive_dict, negative_dict = get_dicts(positive_dir, negative_dir)
    HEADERS = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36', 'Acccept-Language': 'en-US, en; q = 0.5'})

    output_dir = '/content/drive/MyDrive/Internship/Output Data Structure.xlsx'
    output = pd.read_excel(output_dir)

    # Looping over all the URL
    for url in output['URL']:
        
        # hitting the URLs with requests
        webpage = requests.get(url, headers = HEADERS)
        # parsing the response
        soup = BeautifulSoup(webpage.content, 'html.parser')

        try:
            # get the title and content
            title, content = get_data(soup)
            
            # combine them
            content = title + ' ' + content
            
            # save the original text, will be needed down the line
            txt = content
            
            # save the number of fullstops in the original txt
            fullstop_cnt = content.count('.')
            
            # split the content by space as a delimitor
            content = content.split(' ')
            
            # lowercase cast all the words
            content = [word.lower() for word in content]

            # cleaning the content with RESPECT TO THE CUSTOM STOPWORDS (as requested) 
            cleaned_content = list(set(content) - set(stop_words))

            # calculating positive score
            positive_scr = len(set(content).intersection(positive_dict))
            output.loc[output['URL'] == url, 'POSITIVE SCORE'] = positive_scr
            
            # calculating negative score
            negative_scr = len(set(content).intersection(negative_dict))
            output.loc[output['URL'] == url, 'NEGATIVE SCORE'] = negative_scr

            # calculating polarity score
            Polarity_Score = (positive_scr - negative_scr) / ((positive_scr + negative_scr) + 0.000001)
            output.loc[output['URL'] == url, 'POLARITY SCORE'] = Polarity_Score

            # calculating subjectivity score
            subjectivity_score = (positive_scr + negative_scr )/ ( len(cleaned_content) + 0.000001)
            output.loc[output['URL'] == url, 'SUBJECTIVITY SCORE'] = subjectivity_score

            # calculating average sentence length 
            avg_sentence_len = len(cleaned_content) / fullstop_cnt
            output.loc[output['URL'] == url, 'AVG SENTENCE LENGTH'] = avg_sentence_len

            # calculating complex word count
            complex_word_cnt = get_complex(cleaned_content)
            output.loc[output['URL'] == url, 'COMPLEX WORD COUNT'] = complex_word_cnt

            # calculating complex word percentage
            complex_word_per = (complex_word_cnt / len(cleaned_content)) * 100
            output.loc[output['URL'] == url, 'PERCENTAGE OF COMPLEX WORDS'] = complex_word_per
            
            # calculating fog index
            fog_idx = 0.4 * ( avg_sentence_len + complex_word_per )
            output.loc[output['URL'] == url, 'FOG INDEX'] = fog_idx

            # calculating the words per sentence
            words_per_sent = len(cleaned_content) / fullstop_cnt
            output.loc[output['URL'] == url, 'AVG NUMBER OF WORDS PER SENTENCE'] = words_per_sent
            
            
            ## processing the stop words from nltk
            nltk_stp = set(stopwords.words('english'))
            nltk_stp |= set(string.punctuation)
            nltk_stp = [word.lower() for word in nltk_stp]
            filtered_list = [word for word in cleaned_content if word not in nltk_stp]
            word_count = len(filtered_list)
            output.loc[output['URL'] == url, 'WORD COUNT'] = word_count

            ## syllable count per word
            syb_cnt = get_syb(filtered_list) / len(filtered_list)
            output.loc[output['URL'] == url, 'SYLLABLE PER WORD'] = syb_cnt

            ## Personal Pronouns
            count_US = txt.count('US')
            txt = txt.lower()
            pronoun_cnt = txt.count('I') + txt.count('we') + txt.count('ours') + txt.count('my') + txt.count('us') - 2*count_US
            output.loc[output['URL'] == url, 'PERSONAL PRONOUNS'] = pronoun_cnt

            ## Average Word Length
            avg_word_length = len(txt) / len(cleaned_content)
            output.loc[output['URL'] == url, 'AVG WORD LENGTH'] = avg_word_length

        except Exception as e:
          print(f' The following webpage does not exist for {url}') # 3 instances of webpage not founds or error 404 
    # save the output as a excel file ( as requested )
    output.to_excel('Output Data Structure.xlsx', index = False)
 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'NoneType' object has no attribute 'text'
 The following webpage does not exist for https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
'NoneType' object has no attribute 'text'
 The following webpage does not exist for https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
'NoneType' object has no attribute 'text'
 The following webpage does not exist for https://insights.blackcoffer.com/ensuring-growth-through-insurance-technology/
